# NN + Gzip on IMDB Movie Review Dataset

Reimplementation of the pseudocode in the *"Low-Resource" Text Classification: A Parameter-Free Classification Method with Compressors* paper ([https://aclanthology.org/2023.findings-acl.426/](https://aclanthology.org/2023.findings-acl.426/)) 





**Modified to break ties based on choosing the closest neighbors** instead of the lowest index (see explanation in [0_some-concepts.ipynb](0_some-concepts.ipynb)).

<img src="figures/pseudocode.png" width="500">

In [1]:
import gzip
import os.path as op

import numpy as np
import pandas as pd

from local_dataset_utilities import download_dataset, load_dataset_into_to_dataframe, partition_dataset

In [2]:
if not op.isfile("train.csv") and not op.isfile("val.csv") and not op.isfile("test.csv"):
    download_dataset()

    df = load_dataset_into_to_dataframe()
    partition_dataset(df)

In [3]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test.csv")

In [4]:
from tqdm import tqdm
from collections import Counter

k = 2

predicted_classes = []

for row_test in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    test_text = row_test[1]["text"]
    test_label = row_test[1]["label"]
    c_test_text = len(gzip.compress(test_text.encode()))
    distance_from_test_instance = []
    
    for row_train in df_train.iterrows():
        train_text = row_train[1]["text"]
        train_label = row_train[1]["label"]
        c_train_text = len(gzip.compress(train_text.encode()))
        
        train_plus_test = " ".join([test_text, train_text])
        c_train_plus_test = len(gzip.compress(train_plus_test.encode()))
        
        ncd = ( (c_train_plus_test - min(c_train_text, c_test_text))
                / max(c_test_text, c_train_text) )
        distance_from_test_instance.append(ncd)
        
    sorted_idx = np.argsort(np.array(distance_from_test_instance))
    top_k_class = np.array(df_train["label"])[sorted_idx[:k]]
    predicted_class = Counter(top_k_class).most_common()[0][0]
    
    predicted_classes.append(predicted_class)
        
print("Accuracy:", np.mean(np.array(predicted_classes) == df_test["label"].values))

100%|██████████████████████████████████| 10000/10000 [11:40:18<00:00,  4.20s/it]

Accuracy: 0.7191
